## Loading required libraries

In [80]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import load_model

## Reading the data

In [27]:
train_data = pd.read_csv("data/emnist-balanced-train.csv", header = None)
test_data = pd.read_csv("data/emnist-balanced-test.csv", header = None)
train_data.head()
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Separating the response from the predictor variables in the training and testing data.

In [28]:
train_y = train_data[0]
train_y.head()
test_y = test_data[0]
test_y.head()

0    41
1    39
2     9
3    26
4    44
Name: 0, dtype: int64

In [35]:
train_X = train_data.iloc[:, 1:]
train_X.head()
test_X = test_data.iloc[:, 1:]
test_X.head()

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training and testing a simple neural network model

In [84]:
print(train_X.shape)

# # Partitioning the training data
# Xtr, Xts, Ytr, Yts = train_test_split(train_X, train_y, test_size=0.95, random_state=10)
# print(Xtr.shape, Xts.shape, Ytr.shape, Yts.shape)

# use full dataset
Xtr = train_X
Ytr = train_y
print(Xtr.shape, Xts.shape, Ytr.shape, Yts.shape)

(112800, 784)
(112800, 784) (107160, 784) (112800,) (107160,)


### Building the structure of the neural network

In [87]:
# For a single-input model with 47 classes (categorical classification):
num_classes = 47 # number of classes present in the data
inp_dim = train_X.shape[1]
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=inp_dim)) # First hidden layer
model.add(Dense(num_classes, activation='softmax')) # output layer
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Convert the response to one-hot encoding
one_hot_labels = to_categorical(Ytr, num_classes=num_classes)
one_hot_labels

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Training the model

In [88]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(Xtr.values, one_hot_labels, epochs=150, batch_size=32, verbose = 0)

### Testing the model performance on training data

In [101]:
# Accuracy on the training data
pred = model2.predict(Xtr.values)

# taking the maximum output as the predicted class label and then building the confusion matrix
cmat = confusion_matrix(pred.argmax(axis = 1), Ytr.values)
print(cmat)
print("accuracy on training data =", cmat.diagonal().sum()/cmat.sum())

[[   0    0    0 ...,    0    0    0]
 [   0    0    0 ...,    0    0    0]
 [   0    0    0 ...,    0    0    0]
 ..., 
 [   0    0    0 ...,    0    0    0]
 [   3   27    9 ...,   43 2169  468]
 [   0    0    0 ...,    0    0    0]]
accuracy on training data = 0.274875886525


In [98]:
pred.argmax(axis = 1)
Ytr.values

array([45, 36, 43, ..., 23, 31,  8], dtype=int64)

### Saving and reloading the model

In [100]:
model.save('1H_32N_relu.h5')  # creates a HDF5 file '1H_32N_relu.h5'
del model  # deletes the existing model

# returns a compiled model identical to the previous one
model2 = load_model('1H_32N_relu.h5')

## GPU support for Keras

Checking whether Keras backend is using CPU or GPU.

In [81]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4762769127907959993
]
